# Plotting forming limit curves from forming test data

This analysis starts with data exported from the GOM-correlate software.

Each experiment has 3 sections and each section has data for the major and minor strain. Each experiment is repeated multiple times with multiple sample geometries.

We plot 3 forming limit curves (FLC) from the data using different stopping criteria:
* Formation of a crack in the sample
* First derivative necking 
* Second derivative necking.

Formation of a crack in the material is determined by inspection of the sample images.

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain outside the necking region begins to plateau. 

Second derivative necking considers the second derivative of the major strain to determine the necking time. The point at which the second derivative devaites from a constant value is consdiered the necking point.

In [ ]:
from pathlib import Path
import math
import pprint
import glob
from typing import List

import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

First we load the relevent data from the files.

In [ ]:
# Collect all data
def load_data(file_directory: str, fracture_time: int) -> List[np.ndarray]:
    frames = []
    file_list = glob.glob(f"{file_directory}/*.csv")
    for file_name in file_list:
        file_path = Path(file_name)
        data = np.genfromtxt(file_path, skip_header=6, delimiter=";", usecols=[5])
        frames.append(data)
    return frames

major_strain_data = {}
minor_strain_data = {}
for experiment_name, fracture_time in tqdm(fracture_times.items()):
    file_directory = f"../Data/Strain data_All stages/{experiment_name}/Section one/major strain"
    major_strain_data[experiment_name] = load_data(file_directory, fracture_time)    
    
    file_directory = f"../Data/Strain data_All stages/{experiment_name}/Section one/minor strain"
    minor_strain_data[experiment_name] = load_data(file_directory, fracture_time)

## 1). FLC from crack formation

The fracture time of each sample is determined by visual inspection of the sample images at each time step. The fracture times are listed here, one for each sample.

In [ ]:
fracture_times = \
{'Surfalex_10mm_001': 145,
 'Surfalex_10mm_002': 137,
 'Surfalex_10mm_003': 143,
 'Surfalex_20mm_001': 139,
 'Surfalex_20mm_002': 134,
 'Surfalex_20mm_003': 140,
 'Surfalex_40mm_001': 151,
 'Surfalex_40mm_002': 151,
 'Surfalex_40mm_003': 149,
 'Surfalex_60mm_001': 155,
 'Surfalex_60mm_002': 153,
 'Surfalex_60mm_003': 155,
 'Surfalex_120mm_001': 150,
 'Surfalex_120mm_002': 149,
 'Surfalex_120mm_003': 148,
 'Surfalex_full circle_001': 152,
 'Surfalex_fullcircle_002': 157,
 'Surfalex_fullcircle_003': 151
}

In [ ]:
for experiment_name, fracture_time in fracture_times.items():
    # Find the position of the maximum strain as it breaks. 
    # We subtract 2, one to get the frame before fracture and one because the data is zero indexed and the files are 1 indexed.
    max_strain_index = np.argmax(major_strain_data[experiment_name][fracture_time - 2])

    # Collect major strain over time at the fracture position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]

    # Collect the same data but for minor strains
    minor_strain = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]

    plt.plot(minor_strain[:fracture_time - 2], major_strain[:fracture_time - 2], "-x")
plt.xlabel("minor strain")
plt.ylabel("major strain")
plt.title(f"Strain paths for Surflex to crack formation")
plt.savefig("../Results/strain_path_to_break.png", dpi=200)

## 2). FLC from first derivative necking

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain outside the necking region begins to plateau. 
